In [10]:
from IPython.display import display, HTML
import json
import os

import pandas as pd

In [1]:
METHODS = {
    "en": (
        "glossreader_v1",
        "mt0-definition-en-xl",
    ),
    "de": (
        "mt0-definition-en-xl",
        "glossreader_v1",
    ),
    "no1": (
        "mt0-definition-no-xl",
    ),
    "no2": (
        "mt0-definition-no-xl",
    ),
    "ru_ru": (
        "mt0-definition-ru-xl",
    ),
    "no1_en1": (
        "mt0-definition-en-xl",
    ),
    "no2_en2": (
        "mt0-definition-en-xl",
    ),
}

In [ ]:
lang = "en"
annotations_path = os.path.expanduser("annotations/english.json")
with open(annotations_path, "r", encoding="utf8") as f:
    annotations = json.load(f)
gloss_repo = os.path.expanduser("~/PycharmProjects/gloss-annotator")
predictions_folder = os.path.join(gloss_repo, "predictions")
annotations_dict = {}
for sample in annotations:
    # TODO don't uppercase my_text
    word_gloss = sample["data"]["my_text"]
    cluster_1 = sample["data"]["variants"][0]["value"]
    cluster_1_text = sample["data"]["variants"][0]["html"]
    cluster_2 = sample["data"]["variants"][1]["value"]
    cluster_2_text = sample["data"]["variants"][1]["html"]
    print(sample)
    sample_annotations = sample['annotations']
    for annotation in sample_annotations:
        cluster_true = annotation['result'][0]["value"]["choices"][0]
        annotations_dict[(word_gloss, cluster_1, cluster_2)] = (cluster_true, cluster_1_text, cluster_2_text)
        print(cluster_1)
        print(cluster_true)


In [12]:
CLUSTER_NUMBER_COLUMN = 'cluster'

In [47]:
def print_wrongs(method):
    y_true, y_pred = [], []
    print(method)
    for word in os.listdir(os.path.join(predictions_folder,
                                        f"{method}/dwug_{lang}")):
        predictions_path = os.path.join(predictions_folder,
                                        f"{method}/dwug_{lang}/{word}/cluster_gloss.tsv")

        mapping_path = os.path.join(gloss_repo,
                                    f"wugs/label_studio_data/mappings/{lang}/{word}.tsv")
        try:
            mapping = pd.read_csv(mapping_path, sep="\t").astype(str)
        except FileNotFoundError:  # no annotation for this word e.g. if all its clusters are -1 etc.
            continue
        clusters_and_definitions = pd.read_csv(predictions_path, sep="\t")
        clusters_and_definitions[CLUSTER_NUMBER_COLUMN] = \
            clusters_and_definitions[CLUSTER_NUMBER_COLUMN].astype(str)
        clusters_and_definitions = clusters_and_definitions[
            clusters_and_definitions[CLUSTER_NUMBER_COLUMN] != "-1"]

        for row in clusters_and_definitions.iterrows():
            cluster_source, definition = row[1][CLUSTER_NUMBER_COLUMN], row[1][
                "gloss"]
            word_gloss = f"{word.upper()}: <b>{definition.upper()}</b>"
            cluster_filler = mapping[mapping[CLUSTER_NUMBER_COLUMN] == cluster_source][
                "wrong_cluster"]
            if cluster_filler.shape[0]:  # maybe cluster with < 3 usages
                cluster_filler = cluster_filler.iloc[0]
            else:
                continue
            
            annotation = annotations_dict.get((word_gloss, cluster_source, cluster_filler))
            ordering = 0
            if annotation is None:
                annotation = annotations_dict.get((word_gloss, cluster_filler, cluster_source))
                cluster_source_text = annotation[2]
                ordering = 1
            else:
                cluster_source_text = annotation[1]
                
            

            cluster_chosen = annotation[0] # number of cluster chosen by annotator
            
            
            
            
            if cluster_chosen != cluster_source:
                cluster_chosen_text = annotation[2]
                if ordering == 1:
                    cluster_chosen_text = annotation[1]
                display(HTML(word_gloss))
                print("Cluster chosen by annotator:")
                print(cluster_chosen) # -2 fits none, -3 fits both
                display(HTML(cluster_chosen_text))
                print("Cluster that generated the definition")
                print(cluster_source)
                display(HTML(cluster_source_text))
                print("-------------------------------------------------------------------")

In [48]:
print_wrongs("mt0-definition-en-xl")

mt0-definition-en-xl


Cluster chosen by annotator:
-2


Cluster that generated the definition
1


-------------------------------------------------------------------


Cluster chosen by annotator:
1


...

Cluster that generated the definition
2


-------------------------------------------------------------------


Cluster chosen by annotator:
2


Cluster that generated the definition
3


-------------------------------------------------------------------
